Initial cell

In [ ]:
# Downgrade NumPy to fix FAISS compatibility issues
!pip install "numpy<2.0.0"

# Force kernel restart after install
import os
import IPython
os.kill(os.getpid(), 9)


Rerun

In [1]:
import os
from pathlib import Path
from pprint import pprint
import json

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings


PDF

In [2]:
# Set your PDF directory
pdf_dir = Path(".")  # or specify a full path

# Load all PDF files in the directory
pdf_files = list(Path(".").glob("*.pdf"))
docs = [PyPDFLoader(str(pdf)).load() for pdf in pdf_files]

# Trim "References" section if present
for doc in docs:
    content = doc[0].page_content
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]


Chunking (no soya)

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

# Split and flatten chunks
docs_chunks = []
for doc in docs:
    chunks = text_splitter.split_documents(doc)
    docs_chunks.extend(chunks)

# Optional: print basic stats
print(f"Total Chunks: {len(docs_chunks)}")
print("Example chunk:")
print(docs_chunks[0].page_content[:300])


Total Chunks: 466
Example chunk:
© OECD 2017 1
Standard for Automatic Exchange of 
Financial Information in Tax Matters
IMPLEMENTATION  
HANDBOOK
Second edition


FAISS Vector

In [4]:
# Embed using NVIDIA’s cloud embedding endpoint
embedding_model = NVIDIAEmbeddings(model="nvidia/nv-embed-v1")

# Create or update the vector store
db = FAISS.from_documents(docs_chunks, embedding_model)

# Save to disk (optional, for reuse)
db.save_local("faiss_crs_index")


RAG Chain

In [5]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import UnstructuredPDFLoader

# Load vector store
print("Loading FAISS vector store")
db = FAISS.load_local("faiss_crs_index", NVIDIAEmbeddings(model="nvidia/nv-embed-v1"), allow_dangerous_deserialization=True)
retriever = db.as_retriever(search_kwargs={"k": 5})

# Define Mixtral model
llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

# Create a simple prompt template
prompt_template = """Use the following context to answer the question.
If the answer cannot be found in the context, say you don't know.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Build RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


Loading FAISS vector store


Testing

In [8]:
query = "What is the purpose of the Common Reporting Standard (CRS)?"
response = qa_chain.invoke({"query": query})

print("\nCRS Compliance Assistant:\n")
print(response["result"])



CRS Compliance Assistant:

The purpose of the Common Reporting Standard (CRS) is to provide a standardized set of detailed due diligence and reporting rules for financial institutions to follow in order to collect and report information for the automatic exchange of financial information. The CRS ensures consistency in the scope and quality of information exchanged. It contains detailed rules and procedures that financial institutions must follow to ensure the relevant information is collected and reported. These rules must be incorporated into domestic law to ensure due diligence and reporting is performed correctly.


Gradio:

In [ ]:
import gradio as gr
import time

def stream_echo(message, history):
    buffer = ""
    for c in message:
        buffer += c
        time.sleep(0.1)
        yield history + [[message, buffer]]

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Say something...")
    state = gr.State([])

    msg.submit(stream_echo, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(debug=True, share=True)
